In [ ]:
from Funciones import*


In [ ]:
import serial

flagADC = True

# Configura el puerto serial, ajusta el nombre del puerto según sea necesario
puerto_serial = serial.Serial(
    "COM4", 115200
)  # Reemplaza 'COM3' con el nombre de tu puerto


def read_binary_data(serial_port, data_size):
    data = serial_port.read(data_size)  # Lee los datos binarios
    return int.from_bytes(data, byteorder="little")  # Convierte a un entero


data_size = 2
end_of_transmission_char = 0x04

matriz_datos = []


datos_fila = []

    # Escuchar continuamente al puerto serial
while flagADC:
        # Intentar leer un dato
    try:
        linea_serial = read_binary_data(puerto_serial, data_size)
        datos_fila.append(float(linea_serial))
    except Exception as e:
        print(f"Error al leer datos: {e}")
        break

        # Verificar si se ha recibido el carácter especial
    if puerto_serial.in_waiting > 0:
        if ord(puerto_serial.read()) == end_of_transmission_char:
            flagADC= False
            break


In [ ]:
# muestras_test=[249.0, 138.0, 280.0, 130.0, 1027.0, 2516.0, 746.0, 2390.0, 1593.0, 1115.0, 794.0, 522.0, 462.0, 1261.0, 135.0, 4013.0, 745.0, 2034.0, 154.0, 2199.0, 428.0, 449.0, 319.0, 325.0, 693.0, 875.0, 4011.0, 263.0, 1147.0, 3008.0, 146.0, 490.0, 251.0, 189.0, 1900.0, 673.0, 4296.0, 522.0, 347.0, 1592.0, 265.0, 1891.0, 136.0, 1728.0, 681.0, 863.0, 3117.0, 1752.0, 362.0, 725.0, 1322.0, 699.0, 995.0, 564.0, 751.0, 585.0, 379.0, 2144.0, 1187.0, 1793.0, 2600.0, 3398.0, 1542.0, 1577.0, 1196.0, 399.0, 322.0, 3424.0, 298.0, 272.0, 130.0, 215.0, 132.0, 193.0, 129.0, 161.0, 131.0, 156.0, 133.0, 179.0, 137.0, 131.0, 258.0, 129.0, 171.0, 129.0, 240.0, 129.0, 179.0, 127.0, 163.0, 131.0, 154.0, 131.0, 160.0, 137.0, 136.0, 177.0, 132.0, 178.0, 131.0, 166.0, 132.0, 176.0, 131.0, 164.0, 131.0]

# matriz_np = np.array(muestras_test, dtype=float)

# muestras_test = matriz_np.reshape(1, -1)

In [ ]:

muestras_test=[]

muestras_test = datos_fila

In [ ]:
coefs_test = []

wavelet = 'coif6' 
nivel_descomposicion = 3  

for muestras_rango in muestras_test:
    coeficientes = transformadaWavelet(muestras_rango,wavelet,nivel_descomposicion)
    coefs_test.append(coeficientes)


recons_test = []

for coefs in coefs_test:
    senal_recons = reconstruccionWavelet(coefs,wavelet)
    recons_test.append(senal_recons)

datos_analizar = [recons_test]

caract_MAV = calcular_caracteristica_lista(datos_analizar, calcular_MAV)
caract_STD = calcular_caracteristica_lista(datos_analizar, calcular_STD)
caract_RMS = calcular_caracteristica_lista(datos_analizar, calcular_RMS)
caract_VAR = calcular_caracteristica_lista(datos_analizar, calcular_VAR)
caract_WL = calcular_caracteristica_lista(datos_analizar, calcular_WL)
caract_DMAV = calcular_caracteristica_lista(datos_analizar, calcular_DMAV)
caract_SSC = calcular_caracteristica_lista(datos_analizar, calcular_SSC)

X = [caract_MAV,caract_STD,caract_RMS,caract_VAR,caract_WL,caract_DMAV,caract_SSC]

X = np.array(X).reshape(1, -1)

from keras.models import load_model
import numpy as np

# Cargar el modelo desde el archivo
loaded_model = load_model('NN Compiladas/modeloV2_sincoefs_sinnormalizar.h5')

# Realizar predicciones en datos de prueba
predictions = loaded_model.predict(X)

# Obtener las etiquetas predichas como enteros
predicted_labels = np.argmax(predictions, axis=1)

# Mapear los enteros a los nombres de dedos
finger_mapping = {0: 'Dedo_Anular', 1: 'Dedo_Índice', 2: 'Dedo_Medio'}

# Convertir los enteros a nombres de dedos
predicted_fingers = [finger_mapping[label] for label in predicted_labels]

# Imprimir los dedos predichos
print("Dedos predichos:")
print(predicted_fingers)